# Extremely Minimal, Dirty Bleak Scanner implementation 

In [1]:
!pip3 install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
from transitions.extensions.asyncio import AsyncMachine
import asyncio
import time

In [2]:
from bleak import BleakClient, BleakScanner
from bleak.backends.device import BLEDevice

In [3]:
USE_PYCYCLING = False

if USE_PYCYCLING:
    from pycycling.heart_rate_service import HeartRateService


In [4]:
HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID = "00002a37-0000-1000-8000-00805f9b34fb"

In [5]:
class BleakFSMError(Exception):
    '''
    Base class for this library's exceptions.
    When developers use Bleak FSM, they should only see subclasses of this exception.
    No other types of exceptions should pass through and be raised.
    '''
    pass

class StaleScanError(BleakFSMError):
    '''
    Raised when the scan is stale.
    '''
    def __init__(self):
        super().__init__("Scan is stale. Please rescan. Consider setting 'auto_rescan=True' to avoid this error.")

class NoDevicesFoundError(BleakFSMError):
    '''
    Raised when no devices are found during a scan.
    '''
    def __init__(self):
        super().__init__("No devices found during scan.")

In [7]:
class BleakModel:
    '''
    This class is a transitions.AsyncModel wrapper around the BleakScanner class.
    It is used to scan and receive data for Bluetooth Low Energy devices,
    while providing a simple state machine interface to the programmer.

    Many of this class's methods are meant to be state machine callbacks, 
    so they are prefixed with an underscore to prevent accidentally calling them directly.
    '''
    bt_devices = {} # class variable to store the discovered devices, since we can only have one BleakScanner
    _stop_scan_event = asyncio.Event() # class variable to stop the scan
    def __init__(self, scan_stale_time=120, auto_rescan=True, rescan_timeout=3):
        '''
        Args
            (int) scan_stale_time: number of seconds while scan results are considered valid for establishing connection. Must re-scan if connection attempted after this time.
            (bool) auto_rescan: If connect() attempted when scan is stale, run scan for `rescan_timeout` seconds
            (int) rescan_timeout: Number of seconds to scan, if auto_rescan is True. If scan isn't stale, then this value is not used and scan continues indefinitely until you call stop_scan().
        '''
        self.bleak_client: BleakClient = None
        self.ble_device: BLEDevice = None
        self.target = None
        self._stop_streaming_event = asyncio.Event()

        self.scan_stale_time = scan_stale_time
        self.auto_rescan = auto_rescan
        self.rescan_timeout = rescan_timeout

        self.wrapped_client = None # Either identical to `self.bleak_client` (not wrapped) or custom object that represents the BLE device that presumably takes in a BleakClient, such as pycycling classes
        self.enable_notifications = None # an Async Callable must be set later that takes in a BleakClient or similar (Pycycling) object
        self.set_measurement_handler = None  # a Callable must be set later that takes in a BleakClient or similar (Pycycling) object and a value
        self.disable_notifications = None # an Async Callable must be set later that takes in a BleakClient or similar (Pycycling) object

        self.last_scan_time = None
        
    def set_target_address(self, address):
        if address in BleakModel.bt_devices:
            self.target = address
            return True
        else:
            return BleakFSMError(f"Address {address} not found in discovered devices")

    async def _bt_scan(self):        
        def detection_callback(device, _advertisement_data):
            BleakModel.bt_devices[device.address] = device
        async with BleakScanner(detection_callback) as scanner:
            await self._stop_scan_event.wait() # continues to scan until stop_scan_event is set


    async def _stop_bt_scan(self):
        self.last_scan_time = time.time()
        self._stop_scan_event.set()

    async def _connect_to_device(self):
        # stale scan data
        if self.last_scan_time is not None:
            if time.time() - self.last_scan_time > self.scan_stale_time:
                if self.auto_rescan:
                    print("Stale scan. Automatically re-scanning before attempting connect")
                    await self._bt_scan()
                    await asyncio.sleep(self.rescan_timeout)
                    await self._stop_bt_scan()
                    await self._connect_to_device() # while this looks recursive, it isn't because after one iteration, the stale time will not have been reached
                    return True
                else:
                    raise StaleScanError()
                
        if len(BleakModel.bt_devices) == 0:
            raise NoDevicesFoundError()
        try:
            self.ble_device = (BleakModel.bt_devices.pop(self.target))# remove the device from the list to avoid connecting to it multiple times
        except:
            raise BleakFSMError(f"Bluetooth device {self.target} not found in scanned list. It could be powered off, connected to another device, or to another BleakModel.")
        self.bleak_client = BleakClient(self.ble_device) # we don't use the async context manager because we want to access the client object from the disconnect function
        try:
            connected = await self.bleak_client.connect()
            if connected:
                print(f"Connected to {self.target}")

                self.wrapped_client = self.wrap(self.bleak_client)
                    
                return True
            else:
                print(f"Failed to connect to {self.target}")
        except Exception as e:
            print(f"An error occurred: {e}")
            return False
        return True
    
    async def _disconnect_from_device(self):
        BleakModel.bt_devices[self.target] = self.ble_device # put it back in the list
        await self.bleak_client.disconnect()
        print(f"Disconnected from {self.bleak_client.address}")

    async def _stream_from_device(self):
        self._stop_streaming_event.clear()

        self.set_measurement_handler(self.wrapped_client)
        await self.enable_notifications(self.wrapped_client)
        
        print("Started streaming")

        await self._stop_streaming_event.wait()
        return True
    
    async def send_command(self, command):
        if self.state != "Streaming":
            raise BleakFSMError("Cannot send command while not streaming")
        raise NotImplementedError("Todo: Implement this method")

    async def _stop_stream_from_device(self):
        self._stop_streaming_event.set()
        try:
            await self.disable_notifications(self.wrapped_client)

            print("Stopped streaming")
            return True
        except Exception as e:
            print(f"An error occurred while stopping streaming: {e}")
            return False

    async def _stop_stream_and_disconnect_from_device(self):
        '''
        Since we can't re-use the connection after stopping notify,
        we bundle the stop streaming and disconnect logic
        so that the state jumps from Streaming to Init.
        '''
        await self._stop_stream_from_device()
        await self._disconnect_from_device()
        return True
    

In [8]:

model = BleakModel()

if USE_PYCYCLING:
    model.wrap = lambda client: HeartRateService(client)
    model.enable_notifications = lambda client: client.enable_hr_measurement_notifications()
    model.disable_notifications = lambda client: client.disable_hr_measurement_notifications()
    def handle_hr_measurement(value):
        print("Using Pycycling wrapper around BleakClient")
        print(f"Heart Rate: {value}")
    model.set_measurement_handler = lambda client: client.set_hr_measurement_handler(handle_hr_measurement)
else:
    model.wrap = lambda client: client # no wrapping; use the BleakClient object directly
    model.enable_notifications = lambda client: client.start_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID, handle_hr_measurement)
    model.disable_notifications = lambda client: client.stop_notify(HEART_RATE_MEASUREMENT_CHARACTERISTIC_UUID)
    def handle_hr_measurement(sender, data):
        print("Using raw BleakClient")
        print(f"Data received from {sender}: {data}")
        flag = data[0]
        heart_rate = data[1]
        print(f"Heart Rate: {heart_rate} beats per minute")
    model.set_measurement_handler = lambda client: handle_hr_measurement

transitions = []

machine = AsyncMachine(model, states=["Init", "Scanning", "Connected", "Streaming"], transitions=transitions, initial='Init')

machine.add_transition(
    trigger="start_scan",
    source="Init",
    dest="Scanning",
    after="_bt_scan"
)

machine.add_transition(
    trigger="stop_scan",
    source="Scanning",
    dest="Init",
    after="_stop_bt_scan"
)

machine.add_transition(
    trigger="connect",
    source="Init",
    dest="Connected",
    conditions="_connect_to_device",
)

machine.add_transition(
    trigger="stream",
    source="Connected",
    dest="Streaming",
    after="_stream_from_device"
)

# The following two "disconnect" triggers are for
# Connected -> Init (which is obvious) and
# Streaming -> Init: After a stream is stopped, we can't go back to Connected
# because we can't re-use the BleakClient object.
# Therefore we need to go one more back to Init,
# then to Connect, and then back to Streaming.

machine.add_transition(
    trigger="disconnect",
    source="Connected",
    dest="Init",
    before="_disconnect_from_device"
)


machine.add_transition(
    trigger="disconnect",
    source="Streaming",
    dest="Init",
    before="_stop_stream_and_disconnect_from_device"
)

In [9]:
# If we try to run connect() without having scanned, it should fail 
# and raise BleakFSMError
# specifically, `is_device_list_empty` will return True, and the `unless` condition will fail

try:
    await model.connect()
except BleakFSMError as e:
    print(f"Correctly raised error: {e}")

Correctly raised error: No devices found during scan.


In [10]:
# Since we're in a Jupyter notebook, 
# we need to use `create_task()` instead of `get_event_loop().run_until_complete()`
# to use the existing event loop which Jupyter runs on.
asyncio.create_task(model.start_scan())
# if we used
# `await model.start_scan()`
# then it would work, but it would block until the scan is finished, which is not what we want,
# because below we will show how the `model.bt_devices` dictionary gets filled with the results of the scan.

<Task pending name='Task-7' coro=<AsyncEvent.trigger() running at /Users/tensorturtle/Library/Python/3.9/lib/python/site-packages/transitions/extensions/asyncio.py:166>>

In [11]:
model.state

'Scanning'

In [12]:
async def stop_after_delay(delay, model):
    print("Starting sleep for 2 seconds (giving the scanner time to scan)")
    await asyncio.sleep(delay)
    print("Slept for 2 seconds (sending end signal to scanner)")
    await model.stop_scan()
    print("Stopped scanning")
    

asyncio.create_task(stop_after_delay(2, model))

<Task pending name='Task-9' coro=<stop_after_delay() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_73476/921128545.py:1>>

Starting sleep for 2 seconds (giving the scanner time to scan)


In [13]:
async def print_as_we_go():
    for i in range(20):
        print(f"{len(BleakModel.bt_devices)} devices found in {round(i*0.05,2)} seconds.")
        await asyncio.sleep(0.05)

await print_as_we_go()

105 devices found in 0.0 seconds.
105 devices found in 0.05 seconds.
106 devices found in 0.1 seconds.
107 devices found in 0.15 seconds.
107 devices found in 0.2 seconds.
107 devices found in 0.25 seconds.
107 devices found in 0.3 seconds.
109 devices found in 0.35 seconds.
111 devices found in 0.4 seconds.
111 devices found in 0.45 seconds.
111 devices found in 0.5 seconds.
111 devices found in 0.55 seconds.
113 devices found in 0.6 seconds.
113 devices found in 0.65 seconds.
113 devices found in 0.7 seconds.
114 devices found in 0.75 seconds.
114 devices found in 0.8 seconds.
114 devices found in 0.85 seconds.
114 devices found in 0.9 seconds.
114 devices found in 0.95 seconds.


In [14]:
model.bt_devices

{'A9902FBF-BD9D-AD77-924F-9FD7990ED4D0': BLEDevice(A9902FBF-BD9D-AD77-924F-9FD7990ED4D0, Xiaomi Smart Band 8 EFA8),
 'B069CDB8-D98E-A215-1FD5-30D2EA672C54': BLEDevice(B069CDB8-D98E-A215-1FD5-30D2EA672C54, None),
 '2B9EAC64-38BA-12B5-74A7-FF171E753B12': BLEDevice(2B9EAC64-38BA-12B5-74A7-FF171E753B12, None),
 '288A3D7C-D70C-C70A-7A0D-1CF517EBED11': BLEDevice(288A3D7C-D70C-C70A-7A0D-1CF517EBED11, None),
 'D1A5BC49-67F7-F76E-74A9-4FBC1D43392A': BLEDevice(D1A5BC49-67F7-F76E-74A9-4FBC1D43392A, None),
 'DBAD7A59-27E3-663C-9FD2-AFDFE8FF8002': BLEDevice(DBAD7A59-27E3-663C-9FD2-AFDFE8FF8002, None),
 '14863C4D-BF71-4EA3-C6B4-98001056AAF8': BLEDevice(14863C4D-BF71-4EA3-C6B4-98001056AAF8, WHOOPDEDOO),
 'FF88B79E-DE1B-0BA4-EFDE-C03C2CC07141': BLEDevice(FF88B79E-DE1B-0BA4-EFDE-C03C2CC07141, None),
 '8D19A6CB-0CA1-16EF-661C-148809A72E1B': BLEDevice(8D19A6CB-0CA1-16EF-661C-148809A72E1B, None),
 'E1C6E36C-1686-75B1-DAF6-14BD1A1654BD': BLEDevice(E1C6E36C-1686-75B1-DAF6-14BD1A1654BD, 32618441,10,0,0),
 'A

In [15]:
await asyncio.sleep(3)

Slept for 2 seconds (sending end signal to scanner)
Stopped scanning


In [16]:
# Client should select the ID from the following list
target_name = "WHOOPDEDOO"

target_address = ""
for k,v in model.bt_devices.items():
    if v.name == target_name:
        target_address = k

if target_address == "":
    print(f"Fail. No device with name {target_name} found")
else:
    print(f"Success! Found device named {target_name}")

Success! Found device named WHOOPDEDOO


In [20]:
model.set_target_address(target_address)

True

In [21]:
await model.connect() # first time connecting

Connected to 14863C4D-BF71-4EA3-C6B4-98001056AAF8


True

In [22]:
model.state

'Connected'

In [21]:
# a second bluetooth model

model2 = BleakModel()
machine.add_model(model2)

In [22]:
model2.set_target_address("14863C4D-BF71-4EA3-C6B4-98001056AAF8")


__main__.BleakFSMError('Address 14863C4D-BF71-4EA3-C6B4-98001056AAF8 not found in discovered devices')

In [23]:
# since models share the same scan results, this is optional
await model2.start_scan()
asyncio.sleep(3)
await model2.stop_scan()
model2.bt_devices

/var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_84444/1969657322.py:3: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(3)


{'21A2BAB1-5D6C-0EB8-A089-D3BA3EC97555': BLEDevice(21A2BAB1-5D6C-0EB8-A089-D3BA3EC97555, None),
 'DC9EC886-4EE3-4520-3889-F7BDD5458063': BLEDevice(DC9EC886-4EE3-4520-3889-F7BDD5458063, None),
 '680A9629-A8FF-B01B-4065-4CEED62679AA': BLEDevice(680A9629-A8FF-B01B-4065-4CEED62679AA, None),
 'F2284159-6962-7274-C8B8-2E5180CEDCFD': BLEDevice(F2284159-6962-7274-C8B8-2E5180CEDCFD, midea),
 '406672F4-C03E-A1DB-C5AD-9B3050C5CEEE': BLEDevice(406672F4-C03E-A1DB-C5AD-9B3050C5CEEE, None),
 '07A90BAE-DBB9-16F3-E665-93218FF7FF07': BLEDevice(07A90BAE-DBB9-16F3-E665-93218FF7FF07, None),
 'C0C85B6F-FA91-DB58-A8C4-B5A9E4CDE002': BLEDevice(C0C85B6F-FA91-DB58-A8C4-B5A9E4CDE002, None),
 '3FEC3389-7F84-AFC4-6EF1-BDE6CD37F862': BLEDevice(3FEC3389-7F84-AFC4-6EF1-BDE6CD37F862, Green Apple),
 '4786980C-B022-B5D3-AF83-ED5898ECF34B': BLEDevice(4786980C-B022-B5D3-AF83-ED5898ECF34B, None),
 '0581653B-744E-B91D-60DE-38D1D94B45DC': BLEDevice(0581653B-744E-B91D-60DE-38D1D94B45DC, None),
 '18F94F01-45F9-4FA1-B464-5F48A9

In [23]:
async def run_stream():
    await model.stream()

asyncio.create_task(run_stream())

<Task pending name='Task-19' coro=<run_stream() running at /var/folders/16/dbv855p93hx5m7mgrd8fwscw0000gn/T/ipykernel_73476/3697076873.py:1>>

Started streaming
Using raw BleakClient
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00K')
Heart Rate: 75 beats per minute
Using raw BleakClient
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00K')
Heart Rate: 75 beats per minute
Using raw BleakClient
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00K')
Heart Rate: 75 beats per minute
Using raw BleakClient
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00K')
Heart Rate: 75 beats per minute
Using raw BleakClient
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00K')
Heart Rate: 75 beats per minute
Using raw BleakClient
Data received from 00002a37-0000-1000-8000-00805f9b34fb (Handle: 36): Heart Rate Measurement: bytearray(b'\x00K')
Heart 

In [30]:
await asyncio.sleep(10)

Started streaming
HR: HeartRateMeasurement(sensor_contact=False, bpm=67, rr_interval=[1708], energy_expended=None)
HR: HeartRateMeasurement(sensor_contact=False, bpm=66, rr_interval=[488], energy_expended=None)
HR: HeartRateMeasurement(sensor_contact=False, bpm=66, rr_interval=[631], energy_expended=None)
HR: HeartRateMeasurement(sensor_contact=False, bpm=66, rr_interval=[], energy_expended=None)
HR: HeartRateMeasurement(sensor_contact=False, bpm=67, rr_interval=[764, 1133], energy_expended=None)
HR: HeartRateMeasurement(sensor_contact=False, bpm=66, rr_interval=[916], energy_expended=None)
HR: HeartRateMeasurement(sensor_contact=False, bpm=66, rr_interval=[932, 656], energy_expended=None)
HR: HeartRateMeasurement(sensor_contact=False, bpm=66, rr_interval=[], energy_expended=None)
HR: HeartRateMeasurement(sensor_contact=False, bpm=67, rr_interval=[1453], energy_expended=None)
HR: HeartRateMeasurement(sensor_contact=False, bpm=67, rr_interval=[1379], energy_expended=None)


In [24]:
await model.disconnect()

Stopped streaming
Disconnected from 14863C4D-BF71-4EA3-C6B4-98001056AAF8


True

In [25]:
model.state

'Init'